# Task 1: Convolutional Neural Network

To load the data faster I have written the text with its corresponding lable to a csv file. <br>
For Text Classification: I used the Sentiment polarity datasets where 2000 text files are lable as positive or negative polarity .<br>
For Sentence Classification: I used the Subjectivity datasets which contains the rotten_imdb dataset consisting of 5000 subjective and 5000 objective sentences.

Data Preprocessing<br>
For both Text and Sentence classification, <br>
The data is split into 85% to 15% ratio of training set (X_train,Y_train) and testing set(X_test,Y_test).
I have used the label encoder to vectorize the Labels and a tokeniser with max 5000 words to vectorize the text input.
I have then padded the text vectors so that each vector has a constant length of 150.

Models <br>
CNN <br>
The initial layer is the embedding layer. <br>
second layer is a dorpout layer of 0.2<br>
then I have applied a single channel convolutional network with relu activation function.<br>
Followed by a maxpooling with pool size 4 and multi layer perceptron sigmoid activation as an output layer.<br>
Binary cross entropy loss function is used as it is binary classification problem. Adam optimizer is used and accuracy metric is used.<br>

RNN<br>
First layer is the embedding layer.<br>
Then I have implemented a bidirection LSTM layer.<br>
Followed by the a multi layer perceptron with a layer of relu activation <br>
then i have added a dropout layer of 0.2<br>
I have added another multi layer perceptron sigmoid activation as an output layer. <br>
Binary cross entropy loss function is used as it is binary classification problem. Adam optimizer is used and accuracy metric is used.<br>

The Test Accuracies obtained are: <br>
CNN:<br> 
 Text classification : 77.0%<br> 
 Sentence Classification : 88.0%<br>
RNN:<br>
 Text classification : 77.0%<br> 
 Sentence Classification : 89.67%<br>

To improve these accuracies I used Glove vector embeddings. I applied glove vectore embedding for both sentence and text classification with CNN obtained the following test accuracies:<br>
  Sentence classification : 90.93% and <br>
  Text classification : 90.60% <br>


## a. Text Classification

In [2]:
import re
import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D,Bidirectional
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Flatten
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import tensorflow as tf

Using TensorFlow backend.


In [30]:
data_text = pd.read_csv('reviewPolarityCorpus.csv')
df = data_text.sample(frac=1).reset_index(drop=True)
X_text = df["Text"]
Y_text = df["Lable"]
le = LabelEncoder()
Y_text = le.fit_transform(Y_text)
Y_text = Y_text.reshape(-1,1)

X_train,X_test,Y_train,Y_test = train_test_split(X_text,Y_text,test_size=0.15)

max_words = 5000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)


In [31]:
def conv_model():
    model_conv = Sequential()
    model_conv.add(Embedding(max_words, 100, input_length=max_len))
    model_conv.add(Dropout(0.2))
    model_conv.add(Conv1D(64, 5, activation='relu'))
    model_conv.add(MaxPooling1D(pool_size=4))
    model_conv.add(LSTM(100))
    model_conv.add(Dense(1, activation='sigmoid'))
    model_conv.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_conv

In [32]:
model_conv = conv_model()
model_conv.summary()
model_conv.fit(sequences_matrix,Y_train,batch_size=128,epochs=7,
          validation_split=0.2)
scores = model_conv.evaluate(test_sequences_matrix, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100)) 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 150, 100)          500000    
_________________________________________________________________
dropout_8 (Dropout)          (None, 150, 100)          0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 146, 64)           32064     
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 36, 64)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 101       
Total params: 598,165
Trainable params: 598,165
Non-trainable params: 0
_________________________________________________________________
Trai

## b. Sentence Classification

In [6]:
data_sentence = pd.read_csv('rottenImdbCorpus.csv')
data_df = data_sentence.sample(frac=1).reset_index(drop=True)
#data preprocessing
X_sent = data_df["Text"]
Y_sent = data_df["Lable"]
le = LabelEncoder()
Y_sent = le.fit_transform(Y_sent)
Y_sent = Y_sent.reshape(-1,1)

X_train,X_test,Y_train,Y_test = train_test_split(X_sent,Y_sent,test_size=0.15)

max_words = 5000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [7]:
def conv_model():
    model_conv = Sequential()
    model_conv.add(Embedding(max_words, 50, input_length=max_len))
    model_conv.add(Dropout(0.2))
    model_conv.add(Conv1D(64, 5, activation='relu'))
    model_conv.add(MaxPooling1D(pool_size=4))
    model_conv.add(LSTM(100))
    model_conv.add(Dense(1, activation='sigmoid'))
    model_conv.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_conv


In [8]:
model_conv = conv_model()
model_conv.summary()
model_conv.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2)
scores = model_conv.evaluate(test_sequences_matrix, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100)) 


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 150, 50)           250000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 150, 50)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 146, 64)           16064     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 36, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 332,165
Trainable params: 332,165
Non-trainable params: 0
_________________________________________________________________
Trai

## c. Extending the approaches

In [73]:
import numpy as np

x_train,x_test,y_train,y_test = train_test_split(X_sent,Y_sent,test_size=0.15)

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_sent)

x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index


In [74]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 100

x_train = pad_sequences(x_train, padding='post', maxlen=maxlen)
x_test = pad_sequences(x_test, padding='post', maxlen=maxlen)


def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix
embedding_dim = 100
embedding_matrix = create_embedding_matrix('glove.6B/glove.6B.300d.txt',tokenizer.word_index, embedding_dim)

In [75]:
from keras import layers
# create model
model_embedding = Sequential()
model_embedding.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=maxlen))
model_embedding.add(layers.Conv1D(64, 5, activation='relu'))
model_embedding.add(layers.GlobalMaxPooling1D())
model_embedding.add(layers.Dense(10, activation='relu'))
model_embedding.add(layers.Dense(1, activation='sigmoid'))
model_embedding.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])



In [76]:
model_embedding.summary()
history = model_embedding.fit(x_train, y_train,
                    epochs=10,
                    validation_data=(x_test, y_test),
                    batch_size=10)
loss, accuracy = model_embedding.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model_embedding.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy:  {:.4f}".format(accuracy))



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 100, 100)          2300200   
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 96, 64)            32064     
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 64)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                650       
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 11        
Total params: 2,332,925
Trainable params: 2,332,925
Non-trainable params: 0
_________________________________________________________________
Train on 8500 samples, validate on 1500 samples
Epoch 1/10
8500/8500 [==============================] - 36s 4ms/step - loss: 0.3228 

# Task 2: Recurrent Neural Network

## a. Text Classification

In [21]:
data_text = pd.read_csv('reviewPolarityCorpus.csv')
df = data_text.sample(frac=1).reset_index(drop=True)
X_rnn_text = df["Text"]
Y_rnn_text = df["Lable"]
le = LabelEncoder()
Y_rnn_text = le.fit_transform(Y_rnn_text)
Y_rnn_text = Y_rnn_text.reshape(-1,1)

X_train,X_test,Y_train,Y_test = train_test_split(X_rnn_text,Y_rnn_text,test_size=0.15)

max_words = 5000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [78]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = Bidirectional(LSTM(64))(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [79]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

model.fit(sequences_matrix,Y_train,batch_size=128,epochs=7,
          validation_split=0.2)
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

scores = model.evaluate(test_sequences_matrix, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100)) 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 150)               0         
_________________________________________________________________
embedding_19 (Embedding)     (None, 150, 50)           250000    
_________________________________________________________________
bidirectional_6 (Bidirection (None, 128)               58880     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               33024     
_________________________________________________________________
activation_11 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
__________

## b. Sentence Classification

In [7]:
data_sentence = pd.read_csv('rottenImdbCorpus.csv')
data_df = data_sentence.sample(frac=1).reset_index(drop=True)
#data preprocessing
X_rnn_sent = data_df["Text"]
Y_rnn_sent = data_df["Lable"]
le = LabelEncoder()
Y_rnn_sent = le.fit_transform(Y_rnn_sent)
Y_rnn_sent = Y_rnn_sent.reshape(-1,1)

X_train,X_test,Y_train,Y_test = train_test_split(X_rnn_sent,Y_rnn_sent,test_size=0.15)

max_words = 5000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [81]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = Bidirectional(LSTM(64))(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [82]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

model.fit(sequences_matrix,Y_train,batch_size=128,epochs=7,
          validation_split=0.2)
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

scores = model.evaluate(test_sequences_matrix, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100)) 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 150)               0         
_________________________________________________________________
embedding_20 (Embedding)     (None, 150, 50)           250000    
_________________________________________________________________
bidirectional_7 (Bidirection (None, 128)               58880     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               33024     
_________________________________________________________________
activation_13 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
__________

# # c. Extending the approaches 

In [22]:
import numpy as np

x_train,x_test,y_train,y_test = train_test_split(X_rnn_text,Y_rnn_text,test_size=0.15)

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_rnn_text)

x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index


In [23]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 100

x_train = pad_sequences(x_train, padding='post', maxlen=maxlen)
x_test = pad_sequences(x_test, padding='post', maxlen=maxlen)


def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix
embedding_dim = 100
embedding_matrix = create_embedding_matrix('glove.6B/glove.6B.300d.txt',tokenizer.word_index, embedding_dim)

In [12]:
from keras import layers
model_embedding = Sequential()
model_embedding.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=maxlen))
model_embedding.add(layers.Conv1D(64, 5, activation='relu'))
model_embedding.add(layers.GlobalMaxPooling1D())
model_embedding.add(layers.Dense(10, activation='relu'))
model_embedding.add(layers.Dense(1, activation='sigmoid'))
model_embedding.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model_embedding.summary()
history = model_embedding.fit(x_train, y_train,
                    epochs=10,
                    validation_data=(x_test, y_test),
                    batch_size=10)
loss, accuracy = model_embedding.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model_embedding.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy:  {:.4f}".format(accuracy))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 100)          2300200   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 96, 64)            32064     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 2,332,925
Trainable params: 2,332,925
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 8500 samples, validate on 1500 samples
Epoch 